In [2]:
import re

# Node class to represent AST nodes
class Node:
    def __init__(self, node_type, left=None, right=None, value=None):
        self.type = node_type  # 'AND', 'OR', or 'operand'
        self.left = left
        self.right = right
        self.value = value

    def evaluate(self, data):
        if self.type == 'AND':
            return self.left.evaluate(data) and self.right.evaluate(data)
        elif self.type == 'OR':
            return self.left.evaluate(data) or self.right.evaluate(data)
        else:  # Operand node (e.g., 'age >= 30')
            field, operator, val = re.split(r' (>=|<=|>|<|=) ', self.value)
            field = field.strip("() ")

            # Handle string comparisons (strip quotes around string values)
            if val.startswith("'") and val.endswith("'"):
                val = val.strip("'")

            if field in data:
                field_value = data[field]
                print(f"Evaluating: {field} {operator} '{val}' (field_value: {field_value})")  # Debug info
                if isinstance(field_value, str):
                    # String comparisons - make it case insensitive
                    if operator == "=":
                        return field_value.lower() == val.lower()
                    else:
                        return False
                else:
                    # Numeric comparisons
                    field_value = int(field_value)
                    if operator == ">=":
                        return field_value >= int(val)
                    elif operator == "<=":
                        return field_value <= int(val)
                    elif operator == ">":
                        return field_value > int(val)
                    elif operator == "<":
                        return field_value < int(val)
                    elif operator == "=":
                        return field_value == int(val)
                    else:
                        return False
            return False

def create_rule(rule_string):
    rule_string = rule_string.strip()

    if rule_string.startswith("(") and rule_string.endswith(")"):
        return create_rule(rule_string[1:-1])

    match_and = re.search(r'\sAND\s', rule_string)
    match_or = re.search(r'\sOR\s', rule_string)

    if match_and:
        left, right = re.split(r'\sAND\s', rule_string, 1)
        return Node('AND', create_rule(left.strip()), create_rule(right.strip()))
    elif match_or:
        left, right = re.split(r'\sOR\s', rule_string, 1)
        return Node('OR', create_rule(left.strip()), create_rule(right.strip()))
    else:
        return Node('operand', value=rule_string.strip())

def combine_rules(rule_strings):
    """Combines multiple rule strings into a single AST."""
    rules = [create_rule(rule) for rule in rule_strings]
    combined = rules[0]
    for rule in rules[1:]:
        combined = Node('OR', combined, rule)
    return combined

def evaluate_rule(ast, data):
    return ast.evaluate(data)

# Function to get user input for data
def get_user_input():
    age = int(input("Enter age: "))
    department = input("Enter department: ").strip()  
    salary = int(input("Enter salary: "))
    experience = int(input("Enter experience: "))

    return {
        "age": age,
        "department": department,
        "salary": salary,
        "experience": experience
    }

# Main program execution
if __name__ == "__main__":
    # Get user input
    data = get_user_input()

    # Create individual rules based on your examples
    rule1_str = "((age > 30 AND department = 'Sales') OR (age < 25 AND department = 'Marketing')) AND (salary > 50000 OR experience > 5)"
    rule2_str = "((age > 30 AND department = 'Marketing')) AND (salary > 20000 OR experience > 5)"

    # Create AST for individual rules
    rule1_ast = create_rule(rule1_str)
    rule2_ast = create_rule(rule2_str)

    # Combine the rules into a single AST
    combined_ast = combine_rules([rule1_str, rule2_str])

    # Evaluate the rules against user input data
    rule1_result = evaluate_rule(rule1_ast, data)  
    rule2_result = evaluate_rule(rule2_ast, data)  
    combined_result = evaluate_rule(combined_ast, data)  

    # Output the results
    print("Rule 1 Evaluation Result:", rule1_result)  
    print("Rule 2 Evaluation Result:", rule2_result)  
    print("Combined Rule Evaluation Result:", combined_result)  


Evaluating: age > '30' (field_value: 12)
Evaluating: age > '30' (field_value: 12)
Evaluating: age > '30' (field_value: 12)
Evaluating: age > '30' (field_value: 12)
Rule 1 Evaluation Result: False
Rule 2 Evaluation Result: False
Combined Rule Evaluation Result: False


C:\Users\Akilesh\AppData\Local\Temp\ipykernel_20428\1565731880.py:60: DeprecationWarning: 'maxsplit' is passed as positional argument
  left, right = re.split(r'\sAND\s', rule_string, 1)
C:\Users\Akilesh\AppData\Local\Temp\ipykernel_20428\1565731880.py:63: DeprecationWarning: 'maxsplit' is passed as positional argument
  left, right = re.split(r'\sOR\s', rule_string, 1)
